In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [2]:
avisos_detalle = pd.read_csv('../orga-datos/datos_preprocesados/fiuba_6_avisos_detalle.csv')

In [3]:
stopswords="a acá ahí ajena ajeno ajenos ajenas al algo algún algúna algúno algúnos algúnas allá allí ambos ante antes aquel aquella aquello aquellos aquellas aquí arriba así atrás aun aunque bajo bastante bien cabe cada casi cierto cierta ciertos ciertas como con conmigo conseguimos conseguir consigo consigue consiguen consigues contigo contra cual cuales cualquier cualquiera cualquieras cuancuando cuanto cuantas cuanta cuantos de dejar del demás demasiada demasiadas demasiado demasiados dentro desde donde dos e el él ella ellas ello ellos empleáis emplean emplear empleas empleo en encima entonces entre era eras eramos eran eres es esa ese eso esos esas esta estas estaba estado estáis estamos están estar este esto estos estas estoy etc fin fue fueron fui fuimos gueno ha hace haces hacéis hacemos hacen hacer hacia hago hasta incluso intenta intentas intentáis intentamos intentan intentar intento ir jamás junto juntos la lo las los largo más me menos mi mis mía mías mientras mío míos misma mismo mismos mismas modo mucha muchas muchísima muchísimos muchísimo muchísimas mucho muchos muy nada ni ningún ningúna ningúnas ningúno ningúnos no nos nosotras nosotros nuestra nuestro nuestros nuestras nunca o os otra otro otras otros para parecer pero poca poco pocas pocos podéis podemos poder podría podrías podríais podríamos podrían por por qué porque primero puede pueden puedo pues que qué querer quién quiénes quienesquiera quienquiera quizá quizás sabe sabes saben sabéis sabemos saber se según ser si sí siempre siendo sin sino so sobre sois solamente solo sólo somos soy sr sra sres sta su sus suyo suyos suya suyas tal tales también tampoco tan tanta tanto tantos tantas te tenéis tenemos tener tengo ti tiempo tiene tienen todo todos toda todas tomar trabaja trabajo trabajáis trabajamos trabajan trabajar trabajas tras tú tu tus tuya tuyas tuyo tuyos último ultimo un una unas uno unos usa usas usáis usamos usan usar uso usted ustedes va van vais valor vamos varias varios vaya verdadera vosotras vosotros voy vuestra vuestros vuestro vuestras y ya yo como donde requisitos requisito buscamos importante buenas muy grandes beneficios encontramos busqueda zona "
stopswords=stopswords.split()

In [4]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed =[]
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_html_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def foldear_simbolos(cadena):
    ''' 
    Elimina los acentos de una cadena en minúsculas. 
    Ej: descripción -> descripcion
    '''
    acentos = {'á':'a', 'é':'e', 'í':'i', 'ó':'o', 'ú':'u', 'ü':'u'}
    cadena = list(cadena)
    for i in range(len(cadena)):
        if not cadena[i].isalpha():
            cadena[i] = ' '
        else:
            cadena[i] = acentos.get(cadena[i], cadena[i])
    
    return ''.join(cadena)

def es_palabra_inutil(palabra):
    '''
    Devuelve True si palabra es una palabra que no aporta
    contenido.
    '''
    return palabra in set(stopswords)

def parse(descripcion):
    '''
    Normaliza una descripción de aviso del set de datos de Navent.
    Elimina tags HTML, pasa a minúsculas y la convierte a tokens.
    '''
    resultado = []
    
    for palabra in strip_html_tags(descripcion).lower().split():
        if es_palabra_inutil(palabra):
            continue
            
        resultado.extend(foldear_simbolos(palabra).split())
    
    return ' '.join(resultado)
    

In [5]:
words={}
def total_freq(description):
    #it create a dictionary with the following format (word : #appearances in the sum of descriptions)
    words_processed=[]
    for word in description.split():
        if word not in words_processed:
            words[word] = words.get(word, 0) + 1
            words_processed.append(word)
    return description

factor = avisos_detalle["descripcion"].count()*0.01 # %1 aca va el porcentaje que se quiere tener

def remove_less_frequent_words(description):
    # we will remove all the words which appear in less than 1% of the descrptions
    result = []
    for word in description.lower().split():
        if words[word] < factor:
            continue
            
        result.append(word)
    
    return ' '.join(result)

In [6]:
avisos_detalle["descripcion"]=avisos_detalle["descripcion"].map(parse)
avisos_detalle["descripcion"]=avisos_detalle["descripcion"].map(total_freq)
avisos_detalle["descripcion"] = avisos_detalle["descripcion"].map(remove_less_frequent_words)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix

#tf = TfidfVectorizer(input='/home/iglesias/test', analyzer='word', ngram_range=(1,6),
#min_df = 0, stop_words = 'english', sublinear_tf=True)
tf = TfidfVectorizer() 
tfidf_matrix =  tf.fit_transform(avisos_detalle["descripcion"])


feature_names = tf.get_feature_names()

"""doc =0
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print(w, s)
"""
factor=avisos_detalle["descripcion"].count()

scores=[]
dos_mil_palabras_importantes = []
for doc in range(factor):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      if not w in dos_mil_palabras_importantes:   
          scores.append(-s)
          dos_mil_palabras_importantes.append(w)  

import heapq

scores_mas_importantes=[]
heapq.heapify(scores)
puntajes_de_palabras={}
cantidad_de_palabras_que_quiero = 500 #fijarse de tener esta cantidad de palabras sino rompe

for x in range(cantidad_de_palabras_que_quiero):
    scores_mas_importantes.append(heapq.heappop(scores))

dos_mil_palabras_importantes = []    
    
for doc in range(avisos_detalle["descripcion"].count()):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      puntajes_de_palabras[w]=s  
      if -s in scores_mas_importantes:   
          dos_mil_palabras_importantes.append(w) 
            
print("ok")

ok


In [8]:
def mas_impor(description):
    
    resultado = []
    
    for palabra in description.lower().split():
        if palabra not in dos_mil_palabras_importantes:
            continue
            
        resultado.append(palabra)
    
    return ' '.join(resultado)

In [9]:
avisos_detalle["descripcion"]=avisos_detalle["descripcion"].map(mas_impor)

In [10]:
id_av={}

def dic(a):
    l=[]
    for x in range(500):
        l.append(0)
    id_av[a]=l

avisos_detalle["idaviso"].map(dic)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
25258    None
25259    None
25260    None
25261    None
25262    None
25263    None
25264    None
25265    None
25266    None
25267    None
25268    None
25269    None
25270    None
25271    None
25272    None
25273    None
25274    None
25275    None
25276    None
25277    None
25278    None
25279    None
25280    None
25281    None
25282    None
25283    None
25284    None
25285    None
25286    None
25287    None
Name: idaviso, Length: 25288, dtype: object

In [11]:
print(id_av[1001284385])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
word_index={}
contador = [0]
def index_words(description):

    for w in description.split():
        if w not in word_index:
            word_index[w] = contador[0]
            contador[0] = contador[0] +1 
    

In [13]:
avisos_detalle["descripcion"].map(index_words)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
25258    None
25259    None
25260    None
25261    None
25262    None
25263    None
25264    None
25265    None
25266    None
25267    None
25268    None
25269    None
25270    None
25271    None
25272    None
25273    None
25274    None
25275    None
25276    None
25277    None
25278    None
25279    None
25280    None
25281    None
25282    None
25283    None
25284    None
25285    None
25286    None
25287    None
Name: descripcion, Length: 25288, dtype: object

In [14]:
print(len(word_index))

500


In [15]:
test1=avisos_detalle.groupby("idaviso")

In [16]:
for Id,desc in test1:
    l=id_av[Id]
    for w in ((desc["descripcion"].values)[0]).split():
        l[word_index[w]]=l[word_index[w]] + 1*puntajes_de_palabras[w]
    id_av[Id]=l        


In [17]:
print(id_av[1001284385])

[0.15797195624591859, 0.15898117825120481, 0.092402187485583362, 0.081869309615330718, 0.11785002485777586, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [18]:
test1.get_group(1001284385)["descripcion"]

0    estas interesado parte grupo datos
Name: descripcion, dtype: object

In [19]:
print(puntajes_de_palabras["datos"])

0.117850024858


In [20]:
print(word_index["datos"])

4


In [21]:
columns_name=[]
for x in range(500):
    columns_name.append(" ")
for x in word_index:
    columns_name[word_index[x]]=x  

In [22]:
import csv

with open('../orga-datos/words_in_description.csv', 'w') as salida:
        wrt = csv.writer(salida)
        columns_name.insert(0,"idaviso")
        wrt.writerow(columns_name)
        for iD in id_av:
            l=id_av[iD]
            l.insert(0, iD)
            wrt.writerow(l)

In [23]:
test = pd.read_csv('../orga-datos/words_in_description.csv')

In [24]:
test.head()

,idaviso,estas,interesado,parte,grupo,datos,industria,automotriz,candidatos,desempeñarse,...,autogestion,we,you,solutix,lectura,crees,llamadas,pyme,set,mecanicos
0,1112342528,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1111556097,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1112408066,0.0,0.0,0.0,0.0,0.11785,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1112408072,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.069634,0.119199,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1112342538,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
test[test["idaviso"]==1112408066]

,idaviso,estas,interesado,parte,grupo,datos,industria,automotriz,candidatos,desempeñarse,...,autogestion,we,you,solutix,lectura,crees,llamadas,pyme,set,mecanicos
2,1112408066,0.0,0.0,0.0,0.0,0.11785,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
test1.get_group(1112408066)["descripcion"].values

array([ 'orienta operativas area acciones sector venta puntos venta estrategia acciones financieras carga productos datos valorara zona sueldo fijo mas'], dtype=object)